<a href="https://colab.research.google.com/github/RealMyeong/Aiffel_Exploration/blob/main/EX_BS2_project6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1. 데이터 다운로드

데이터 직접 다운로드 받아서 구글 드라이브에 저장함

# Step 2. 데이터 읽어오기

In [1]:
import glob

txt_file_path = '/content/drive/MyDrive/AIFFEL/EX/6. 작사가 만들기/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

#여러개의 txt파일을 모두 읽어서 row_corpus에 저장
for txt_file in txt_list:
  with open(txt_file, 'r') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)

print('데이터 크기: ',len(raw_corpus))
print('Examples:\n', raw_corpus[:3])

데이터 크기:  187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


# Step 3. 데이터 정제

앞서 배운 테크닉들을 활용해 문장 생성에 적합한 모양새로 데이터를 정제하세요!

preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.

In [2]:
import tensorflow as tf
import re

def preprocess_sentence(s):
  s = s.lower().strip() #1 문장을 소문자로 바꾸고 양쪽 공백을 지움
  s = re.sub(r"([?.!,¿])", r" \1", s) #2 특수 문자 양쪽에 공백을 넣음
  s = re.sub(r'[" "]+', " ", s) #3 여러개의 공백을 하나의 공백으로 바꿈
  s = re.sub(r'[^a-zA-Z?.!,¿]+', ' ', s) #4 a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿈
  s = s.strip() #5 양쪽 공백을 지움
  s = '<start> '+s+' <end>' #6 문장 시작에 <start> 끝에 <end> 넣어줌
  return s

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[1]

'<start> made my way into the night <end>'

In [4]:
def tokenize(corpus):
  tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=15000,
                                                    filters=' ',
                                                    oov_token='<unk>')
  tokenizer.fit_on_texts(corpus)
  tensor = tokenizer.texts_to_sequences(corpus)
  tensor_2 = []
  print(tensor[0])
  for i in range(len(tensor)): #토큰 개수가 15개 이하만 가져옴
    if len(tensor[i]) <= 15:
      tensor_2.append(tensor[i])

  tensor_2 = tf.keras.preprocessing.sequence.pad_sequences(tensor_2, padding='post')
  

      
  print(tensor_2, tokenizer)
  return tensor_2, tokenizer

tensor_2, tokenizer = tokenize(corpus)

[2, 303, 28, 99, 4825, 3]
[[  2 303  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]
 [  2   3   0 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fe294baa250>


In [5]:
type(tensor_2)

numpy.ndarray

In [6]:
tensor_2.shape

(156074, 15)

In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : to
10 : a


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성
src_input = tensor_2[:, :-1]  

# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor_2[:, 1:]    

print((src_input).shape)
print((tgt_input).shape)

(156074, 14)
(156074, 14)


# Step 4. 평가 데이터셋 분리

훈련 데이터와 평가 데이터를 분리하세요!

tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(src_input,
                                                    tgt_input,
                                                    test_size = 0.2,
                                                    random_state = 27)

# Step 5. 인공지능 만들기

In [10]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듦
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset element_spec=(TensorSpec(shape=(256, 14), dtype=tf.int32, name=None), TensorSpec(shape=(256, 14), dtype=tf.int32, name=None))>

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_3 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.rnn_3(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 512
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-4.73902037e-05,  2.18599962e-05, -5.16655155e-06, ...,
         -6.23064188e-05, -3.35342884e-06, -4.60978208e-06],
        [-1.01163838e-04,  1.10990506e-04,  2.56117874e-05, ...,
         -1.23248275e-04,  4.54557921e-05,  1.28942493e-05],
        [-9.78249736e-05,  2.46599404e-04,  7.26079234e-05, ...,
         -1.84746285e-04,  1.33556736e-04,  6.33578238e-05],
        ...,
        [ 2.32638326e-04, -6.20293897e-04, -7.16078444e-04, ...,
         -4.02541016e-04,  7.66858226e-04, -1.23077445e-03],
        [ 1.55132817e-04, -7.73789769e-04, -9.52937815e-04, ...,
         -4.39250260e-04,  8.77662504e-04, -1.53662753e-03],
        [ 6.24772292e-05, -9.10923351e-04, -1.19054900e-03, ...,
         -4.67473437e-04,  9.79109202e-04, -1.83556241e-03]],

       [[-4.73902037e-05,  2.18599962e-05, -5.16655155e-06, ...,
         -6.23064188e-05, -3.35342884e-06, -4.60978208e-06],
        [-1.25385573e-04,  4.15759096e-05,  1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  7680512   
                                                                 
 lstm (LSTM)                 multiple                  6295552   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 lstm_2 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  15376025  
                                                                 
Total params: 46,137,497
Trainable params: 46,137,497
Non-trainable params: 0
_________________________________________________________________


In [14]:

optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)


model.fit(X_train, y_train, batch_size=256, validation_data=(X_test, y_test), epochs=10)

Epoch 1/10
488/488 [==============================] - 79s 151ms/step - loss: 3.5397 - val_loss: 3.2034
Epoch 2/10
488/488 [==============================] - 72s 148ms/step - loss: 3.0783 - val_loss: 3.0378
Epoch 3/10
488/488 [==============================] - 72s 148ms/step - loss: 2.9465 - val_loss: 2.9672
Epoch 4/10
488/488 [==============================] - 72s 148ms/step - loss: 2.8554 - val_loss: 2.9094
Epoch 5/10
488/488 [==============================] - 76s 157ms/step - loss: 2.7766 - val_loss: 2.8601
Epoch 6/10
488/488 [==============================] - 76s 157ms/step - loss: 2.7036 - val_loss: 2.8204
Epoch 7/10
488/488 [==============================] - 76s 156ms/step - loss: 2.6365 - val_loss: 2.7874
Epoch 8/10
488/488 [==============================] - 76s 157ms/step - loss: 2.5741 - val_loss: 2.7581
Epoch 9/10
488/488 [==============================] - 72s 148ms/step - loss: 2.5145 - val_loss: 2.7348
Epoch 10/10
488/488 [==============================] - 76s 157ms/step - l

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i hate")

'<start> i hate the way you lie <end> '